In [ ]:
'''
-Creación de instancia en EC2 de AWS, de manera segura
Para crear la capa gratuita, primero se debe crear una cuenta en la página de Amazon Web Services a través de
https://aws.amazon.com/es/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all
siguiendo los pasos y confirmando correo.
Teniendo esto listo, se realizan los requisitos previos mediante https://docs.aws.amazon.com/AWSEC2/latest/WindowsGuide/get-set-up-for-amazon-ec2.html
donde se crean los "Key pair" y "Security group".
Luego, usando la guia en https://docs.aws.amazon.com/AWSEC2/latest/WindowsGuide/EC2_GetStarted.html, donde primero se crea
una instancia usando parametros especificos para Windows (en este caso), y luego conectar a la instancia mediante un archivo .pem
que entrega una contraseña encriptada para acceder a la instancia.

-Creación de Base de Datos con autenticación

Para esta etapa, se usará la guia disponible en https://docs.aws.amazon.com/quickstart/latest/mongodb/deployment.html,
donde se crea un entorno de MongoDB que utilice la instancia EC2 creada anteriormente, por lo que es conveniente seguir los pasos
de creación en https://docs.aws.amazon.com/quickstart/latest/mongodb/step2.html para VPC existentes. En general, si se mantienen
los parámetros por defecto se crearan los nodos sin problemas, siendo el estado del stack lo que más demora en completar, puesto que 
hay muchas variables de por medio. Al terminar con un estado "CREATE_COMPLETE" se crearán varios nodos de MongoDB como instancias EC2 en AWS.


Creación de usuario para revisar la información persistida

La guía anterior fue diseñada para usuarios de Linux, por lo que la etapa 3 sobre conexión a nodos de MongoDB mediante SSH 
no es posible de realizar en Windows. Sin embargo, usando el software Studio 3T, que es un IDE de MongoDB, es posible conectarse
a los nodos mediante la guía disponible en https://setu677.medium.com/how-to-connect-to-mongodb-on-aws-ec2-instance-with-robomongo-b2977b8a112f.
Pero esto no garantiza la creación de un usuario para acceder a la base de datos conectada.
Ahora bien, para el caso de Linux, siguiendo la guia disponible en https://medium.com/founding-ithaka/setting-up-and-connecting-to-a-remote-mongodb-database-5df754a4da89
es posible crear un usuario primero abriendo el shell de Mongo en la terminal (suponiendo que MongoDB está instalado) y 
colocando el siguiente código:
ubuntu:~$ mongo
> use admin;
> db.createUser({
      user: "admin",
      pwd: "myadminpassword",
      roles: [
                { role: "userAdminAnyDatabase", db: "admin" },
                { role: "readWriteAnyDatabase", db: "admin" },
                { role: "dbAdminAnyDatabase",   db: "admin" }
             ]
  });
> db.createUser({
      user: "user1",
      pwd: "user1password",
      roles: [
                { role: "userAdmin", db: "sampledb" },
                { role: "dbAdmin",   db: "sampledb" },
                { role: "readWrite", db: "sampledb" }
             ]
  });
Luego se le permite a MongoDB tener acceso a estos controles de acceso y además garantizar acceso al puerto (27017) en la instancia EC2.
Una vez hecho esto, se pueden probar las conexiones a la base de datos usando la libreria Pymongo en Python y establecer un pipeline de forma
que los datos obtenidos por el Crawler de Scrapy se guarden remotamente en MongoDB.
'''